In [4]:
import math
import numpy as np


class Graph:
    def __init__(self, graph_struct = {}):
        self.graph = graph_struct

    def __str__(self):
        grh = ''
        for vrt in self.getVertices():
            for adj in self.getAdjacent(vrt):
                grh += '({0}, {1}, {2})\t'.format(vrt, adj, self.graph[vrt][adj])
        return grh

    def setVertex(self, vertex):
        if vertex not in self.graph.keys():
            self.graph[vertex] = {}
        return self

    def setAdjacent(self, vertex, adj, weight = 0):
        if vertex not in self.graph.keys():
            self.graph[vertex] = {}
        if adj not in self.graph.keys():
            self.graph[adj] = {}
        
        self.graph[vertex][adj] = weight

        return self
    
    def getVertices(self):
        return list(self.graph.keys())

    def getAdjacent(self, vertex):
        if vertex in self.graph.keys():
            return self.graph[vertex]

    def getPathCost(self, path):
        pathCost = 0
        for vrt, adj in zip(path, path[1:]):
            pathCost += self.graph[vrt][adj]
        return pathCost


class GeneticAlgorithmTSP:
    def __init__(self, generations=100, population_size=10, tournamentSize=4, mutationRate=0.1, elitismRate=0.1):
        self.generations = generations
        self.population_size = population_size
        self.tournamentSize = tournamentSize
        self.mutationRate = mutationRate
        self.elitismRate = elitismRate
    
    def optimize(self, graph):
        population = self.__makePopulation(graph.getVertices())
        elitismOffset = math.ceil(self.population_size*self.elitismRate)

        if (elitismOffset > self.population_size):
            raise ValueError('Elitism Rate must be in [0,1].')
        
        print ('Optimizing TSP Route for Graph:\n{0}'.format(graph))

        for generation in range(self.generations):
            print ('\nGeneration: {0}'.format(generation + 1))
            print ('Population: {0}'.format(population))
            
            newPopulation = []            
            fitness = self.__computeFitness(graph, population)
            print ('Fitness:    {0}'.format(fitness))
            fittest = np.argmin(fitness)

            print ('Fittest Route: {0} ({1})'.format(population[fittest], fitness[fittest]))
            
            if elitismOffset:
                elites = np.array(fitness).argsort()[:elitismOffset]
                [newPopulation.append(population[i]) for i in elites]
            for gen in range(elitismOffset, self.population_size):
                parent1 = self.__tournamentSelection(graph, population)
                parent2 = self.__tournamentSelection(graph, population)
                offspring = self.__crossover(parent1, parent2)
                newPopulation.append(offspring)
                # print ('\nParent 1: {0}'.format(parent1))
                # print ('Parent 2: {0}'.format(parent2))
                # print ('Offspring: {0}\n'.format(offspring))
            for gen in range(elitismOffset, self.population_size):
                newPopulation[gen] = self.__mutate(newPopulation[gen])
    
            population = newPopulation

            if self.__converged(population):
                print ("\nConverged to a local minima.", end = " ")
                break

        return (population[fittest], fitness[fittest])


    def __makePopulation(self, graph_nodes):
        l =  [''.join(v for v in np.random.permutation(graph_nodes)) for i in range(self.population_size)]
        for i in range(len(l)) :
            l[i] = l[i] + l[i][0]
        return l

    def __computeFitness(self, graph, population):
        return [graph.getPathCost(path) for path in population]


    def __tournamentSelection(self, graph, population):
        tournament_contestants = np.random.choice(population, size=self.tournamentSize)
        # print (tournament_contestants)
        tournament_contestants_fitness = self.__computeFitness(graph, tournament_contestants)
        return tournament_contestants[np.argmin(tournament_contestants_fitness)]
    

    def __crossover(self, parent1, parent2):
        offspring = ['' for allele in range(len(parent1))]
        index_low, index_high = self.__computeLowHighIndexes(parent1)
        
        offspring[index_low:index_high+1] = list(parent1)[index_low:index_high+1]
        offspring_available_index = list(range(0, index_low)) + list(range(index_high+1, len(parent1)))        
        for allele in parent2:
            if '' not in offspring:
                break
            if allele not in offspring:
                offspring[offspring_available_index.pop(0)] = allele
        l = ''.join(v for v in offspring)
        l = l + l[0]
        return l; 


    def __mutate(self, genome):
        if np.random.random() < self.mutationRate:
            index_low, index_high = self.__computeLowHighIndexes(genome)
            return self.__swap(index_low, index_high, genome)
        else:
            return genome


    def __computeLowHighIndexes(self, string):
        index_low = np.random.randint(0, len(string)-1)
        index_high = np.random.randint(index_low+1, len(string))
        while index_high - index_low > math.ceil(len(string)//2):
            try:
                index_low = np.random.randint(0, len(string))
                index_high = np.random.randint(index_low+1, len(string))
            except ValueError:
                pass
        return (index_low, index_high)


    def __swap(self, index_low, index_high, string):
        string = list(string)
        string[index_low], string[index_high] = string[index_high], string[index_low]
        return ''.join(string)


    def __converged(self, population):
        return all(genome == population[0] for genome in population)


if __name__ == '__main__':
    graph = Graph()
    graph.setAdjacent('A', 'B', 4)
    graph.setAdjacent('A', 'C', 12)
    graph.setAdjacent('A', 'D', 7)
    graph.setAdjacent('B', 'A', 5)
    graph.setAdjacent('B', 'D', 18)
    graph.setAdjacent('C', 'A', 11)
    graph.setAdjacent('C', 'D', 6)
    graph.setAdjacent('D', 'A', 10)
    graph.setAdjacent('D', 'B', 2)
    graph.setAdjacent('D', 'C', 3)
    graph.setAdjacent('B', 'C', 999)
    graph.setAdjacent('C', 'B', 999)


    ga_tsp = GeneticAlgorithmTSP(generations = 20, population_size = 7, tournamentSize = 2, mutationRate = 0.2, elitismRate = 0.1)
    
    optimal_path, path_cost = ga_tsp.optimize(graph)
    print ('\nPath: {0}, Cost: {1}'.format(optimal_path, path_cost))

Optimizing TSP Route for Graph:
(A, B, 4)	(A, C, 12)	(A, D, 7)	(B, A, 5)	(B, D, 18)	(B, C, 999)	(C, A, 11)	(C, D, 6)	(C, B, 999)	(D, A, 10)	(D, B, 2)	(D, C, 3)	

Generation: 1
Population: ['CBDAC', 'CABDC', 'DACBD', 'ADCBA', 'ACBDA', 'CDBAC', 'BCDAB']
Fitness:    [1039, 36, 1039, 1014, 1039, 25, 1019]
Fittest Route: CDBAC (25)

Generation: 2
Population: ['CDBAC', 'ADBCA', 'BDACB', 'CBDAC', 'CDABC', 'ADCBA', 'ADCBA']
Fitness:    [25, 1019, 1039, 1039, 1019, 1014, 1014]
Fittest Route: CDBAC (25)

Generation: 3
Population: ['CDBAC', 'CDBAC', 'ACBDA', 'CDBAC', 'DCBAD', 'CDABC', 'BDCAB']
Fitness:    [25, 25, 1039, 25, 1014, 1019, 36]
Fittest Route: CDBAC (25)

Generation: 4
Population: ['CDBAC', 'CDABC', 'CBDAC', 'DCBAD', 'CADBC', 'CDBAC', 'DCABD']
Fitness:    [25, 1019, 1039, 1014, 1019, 25, 36]
Fittest Route: CDBAC (25)

Generation: 5
Population: ['CDBAC', 'CBACD', 'CBADC', 'DBACD', 'DBACD', 'DCBAD', 'DBACD']
Fitness:    [25, 1022, 1014, 25, 25, 1014, 25]
Fittest Route: CDBAC (25)

Genera